In [5]:
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_groq import ChatGroq
from langchain.schema import Document  # Import Document class
import requests

In [11]:
groq_api_key = "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"
audio_url = "https://public-v1-stg.omicrm.com/third_party/recording/uc?id=ZkJDTGhDTk1CeWZUUFZqYy8zS2x0QnJLeDhUT29pSGZoVExpK21tRStrK202dFJldVp5Q0ZwQSsvY2poaE9FeG80cnV1bkRJVkRMSnlXK0g0MldRVEE9PQ==&code=c06e4571-2698-46db-af03-23c8da0e73f6" 

In [12]:
client = Groq(api_key = groq_api_key)
model = 'whisper-large-v3-turbo'
summarizer_model = ChatGroq(api_key=groq_api_key, model="Gemma-7b-It")

In [2]:

def fetch_audio(audio_url):
    """Fetch audio data from a given URL."""
    response = requests.get(audio_url)
    if response.status_code == 200:
        with open("temp_audio.mp3", "wb") as f:
            f.write(response.content)
        return "temp_audio.mp3"
    else:
        raise Exception("Failed to download audio file.")

In [7]:
def audio_to_text(filepath):
    with open(filepath, "rb") as file:
        translation = client.audio.transcriptions.create(
            file=(filepath, file.read()),
            model="whisper-large-v3",
        )
    return translation.text

In [42]:
def summarize_text(text):
    """Summarize text using LangChain with GROQ's summarization model."""
    # Define the summarization prompt
    prompt_template = """
    Cung cấp nội dung của đoạn hội thoại trong phạm vi 300 từ:
    Nội dung: {text}
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["text"]
    )
    
    # Wrap text in a Document object
    document = Document(page_content=text)

    # Set up the summarization chain
    chain = load_summarize_chain(summarizer_model, chain_type="stuff", prompt=prompt)
    summary = chain.run([document])

    return summary

In [14]:
audio_file_path = fetch_audio("https://public-v1-stg.omicrm.com/third_party/recording/uc?id=UHpuUTl2ZXcvamlIeFpDdkVaUDRRVkVMMVRIeXQvbStKQ0JFUFVwYTdjKy9rRmUyWnBkdTg2TDNYM09IaFVST280cnV1bkRJVkRMSnlXK0g0MldRVEE9PQ==&code=3a9cd43a-9aed-46f4-9daf-510d1c1fad8f")
translation_text = audio_to_text("temp_audio.mp3")
##summary = summarize_text(translation_text)
##print(summary)